In [ ]:
# Check GPU
!nvidia-smi


!pip install imbalanced-learn

# Install RAPIDS
!pip install --upgrade pip
!pip install cudf-cu11 cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com

In [ ]:
# Data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# File and OS operations
import os
import glob

# Google Colab files (if using Colab)
from google.colab import files

# Warnings and display
import warnings
from IPython.display import display

# Scipy statistics
from scipy.stats import ttest_ind

# Scikit-learn modules
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    train_test_split,
    learning_curve,
    cross_val_score,
    cross_val_predict
)
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_recall_curve,
    average_precision_score,
    accuracy_score,
    log_loss,
    f1_score,
    mean_squared_error,
    precision_score,
    recall_score,
    make_scorer
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

# XGBoost
from xgboost import XGBClassifier

# Imbalanced-learn modules
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

# Joblib for model persistence
import joblib


In [ ]:
from sklearn import __version__ as sklearn_version
print("scikit‑learn version:", sklearn_version)


Code to fuse data

In [ ]:
uploaded_with_prep = files.upload()

fused_with_prep = pd.DataFrame()

for filename, content in uploaded_with_prep.items():
    temp_df = pd.read_csv(filename)
    fused_with_prep = pd.concat([fused_with_prep, temp_df], ignore_index=True)

# Save and download fused DataFrames
fused_with_prep.to_csv('extracted_features_with_prep.csv', index=False)

print("Fused files created:")
#files.download('extracted_features_with_prep.csv')


Visualize data and Clean Names and Features

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
file_path = "extracted_features_with_prep.csv"
df = pd.read_csv(file_path)

# Fix Names of Features
df.columns = [col.replace("\n ", " ").strip() for col in df.columns]

# Save the Cleaner dataset
df.to_csv("extracted_features_with_prep.csv", index=False)

plt.figure(figsize=(15, 10))
df.hist(bins=30, figsize=(20, 15), edgecolor='black')
plt.suptitle("Feature Distributions", fontsize=16)
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Set a clean style
sns.set_theme(style="whitegrid")

# 1) Count Plot for Gender
plt.figure(figsize=(6, 4))
sns.countplot(
    data=df,
    x='Gender',
    palette='Set2',
    edgecolor='black',
    linewidth=0.5
)
plt.title("Gender Distribution")
plt.xticks([0, 1], ["Male (1)", "Female (2)"])
plt.xlabel("Gender")
plt.ylabel("Count")
plt.show()

# 2) Histogram for Age
min_age = df['Age'].min()
max_age = df['Age'].max()
bins = np.arange(min_age - 0.5, max_age + 0.5 + 1, 1)

plt.figure(figsize=(6, 4))
sns.histplot(
    data=df,
    x='Age',
    bins = np.arange(min_age - 0.5, max_age + 0.5 + 1, 1),
    palette='Set2',
    edgecolor='black',
    linewidth=0.5
)
plt.title("Age Distribution")
plt.xticks(np.arange(min_age, max_age + 1, 1))
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()


Sort the left vs right status sorting

In [ ]:
# Function to update statuses based on filename and status
def update_status(df):
    for index, row in df.iterrows():
        filename = row['Filename']
        status = row['Status']

        # Ensure filename is long enough to access the 8th character
        if len(filename) >= 8:
            eighth_char = filename[7]  # Python indexing starts at 0

            # Apply conditions based on the rules
            if status == 1:
                if eighth_char == 'R':
                    df.at[index, 'Status'] = 0
                elif eighth_char == 'L':
                    df.at[index, 'Status'] = 1
            elif status == 2:
                if eighth_char == 'R':
                    df.at[index, 'Status'] = 1
                elif eighth_char == 'L':
                    df.at[index, 'Status'] = 0
            elif status == 3:
                if eighth_char == 'R' or eighth_char == 'L':
                    df.at[index, 'Status'] = 1
    return df


# Function to update measurement columns based on the leg indicated in the filename
def update_measurements(df):
    def select_measurements(row):
        filename = row['Filename']
        if len(filename) >= 8:
            leg = filename[7]  # 8th character: 'L' or 'R'
        else:
            leg = None

        if leg == 'L':
            return pd.Series({
                "Leg Length": row["L Leg Length"],
                "Q-Angle": row["L Q-Angle"],
                "Active Knee": row["L Active Knee Flexion Angle"]
            })
        elif leg == 'R':
            return pd.Series({
                "Leg Length": row["R Leg Length"],
                "Q-Angle": row["R Q-Angle"],
                "Active Knee": row["R Active Knee Flexion Angle"]
            })
        else:
            return pd.Series({"Leg Length": None, "Q-Angle": None, "Active Knee": None})

    # Apply row-wise selection and create new columns
    df[['Leg Length', 'Q-Angle', 'Active Knee']] = df.apply(select_measurements, axis=1)

    # Drop the original left/right specific measurement columns
    df.drop(columns=['L Leg Length', 'R Leg Length', 'L Q-Angle', 'R Q-Angle',
                     'L Active Knee Flexion Angle', 'R Active Knee Flexion Angle',], inplace=True)
    return df

# Load the dataset
file_path = "extracted_features_with_prep.csv"
df = pd.read_csv(file_path)

update_status(df)
update_measurements(df)

# Save the cleaned dataset
df.to_csv("processed_features.csv", index=False)


plt.figure(figsize=(15, 10))
df.hist(bins=30, figsize=(20, 15), edgecolor='black')

plt.suptitle("Feature Distributions", fontsize=16)
plt.show()


# Overwrite the existing CSV file with the modified DataFrame
df.to_csv("extracted_features_with_prep.csv", index=False)


drop code number

In [ ]:
df.drop('Code_Number_6digits', axis=1, inplace=True)
df.to_csv("extracted_features_with_prep.csv", index=False)



In [ ]:
files.download('extracted_features_with_prep.csv')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = "extracted_features_with_prep.csv"
df = pd.read_csv(output_combined_file)

# Create the boxplot on its own Axes
fig, ax = plt.subplots(figsize=(30, 10))
sns.boxplot(data=df, whis=1.5, width=0.5, ax=ax)

# Customize titles and tick labels
ax.set_title("Feature Distributions", fontsize=16)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Make layout tight so labels and boxes don't overlap
plt.tight_layout()
plt.show()

# (Optional) Overwrite the CSV if you've made in‑memory modifications
df.to_csv(file_path, index=False)

# Print out the column names
print("Columns in DataFrame:", df.columns.tolist())


In [ ]:
# Define the features to plot against "Status"
features = [
      'Mean', 'Std', 'Variance', 'Skewness', 'Kurtosis',
       'Dissimilarity_0', 'Dissimilarity_45', 'Dissimilarity_90',
       'Dissimilarity_135', 'Correlation_0', 'Correlation_45',
       'Correlation_90', 'Correlation_135', 'Homogeneity_0', 'Homogeneity_45',
       'Homogeneity_90', 'Homogeneity_135', 'Energy_0', 'Energy_45',
       'Energy_90', 'Energy_135', 'Contrast_0', 'Contrast_45', 'Contrast_90',
       'Contrast_135', 'RunLengthNonUniformity', 'ShortRunEmphasis',
       'LongRunEmphasis', 'GrayLevelNonUniformity', 'RunPercentage',
       'RunEntropy', 'Area', 'Eccentricity', 'Perimeter', 'Centroid_X',
       'Centroid_Y', 'Hu0',	'Hu1',	'Hu2',	'Hu3',	'Hu4',	'Hu5',	'Hu6', 'Age',
       'Gender', 'Active Playing Years', 'Height', 'Weight',
       'Leg Length', 'Q-Angle', 'Active Knee'
            ]

# Function to create scatter plots for all features in a single figure
def plot_scatter_grid(scaled_df, title):
    num_features = len(features)
    num_cols = 5
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)  # Calculate rows

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 15))
    fig.suptitle(title, fontsize=18)

    axes = axes.flatten()  # For easy iteration

    for i, feature in enumerate(features):
        sns.scatterplot(ax=axes[i], data=scaled_df, x=feature, y='Status', alpha=0.5)
        axes[i].set_title(feature)
        axes[i].set_xlabel("")
        axes[i].set_ylabel("Status")

    # Hide any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit title
    plt.show()

plot_scatter_grid(df, "Raw Data - Scatter Plots")

Initial Normalization for viewing

In [ ]:
# Load the uploaded CSV file into a DataFrame
df_with_prep = pd.read_csv('extracted_features_with_prep.csv')


def normalize_features_min_max(df):
    normalized_df = df.copy()
    for column in normalized_df.columns:
        # Skip non-numeric columns
        if column in ['Filename', 'Status', 'Gender']:
            continue

        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(normalized_df[column]):
            min_val = normalized_df[column].min()
            max_val = normalized_df[column].max()

            # Avoid division by zero (constant column)
            if max_val != min_val:
                normalized_df[column] = (normalized_df[column] - min_val) / (max_val - min_val)
            else:
                print(f"Warning: Column '{column}' has constant values. Skipping normalization.")
    return normalized_df


df_with_prep_normalized = normalize_features_min_max(df_with_prep)


df_with_prep_normalized.to_csv('Features_With_Preprocessing_Normalized.csv', index=False)
print("Normalized 'with_prep' file saved.")

spearman methods

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

# Load and clean data.
df = pd.read_csv("extracted_features_with_prep.csv")
df = df.drop(columns=[col for col in df.columns if "Code_Number_6digits" in col])
#df = df.drop(columns=["Filename"])

# Extract features and target.
X = df.drop(columns=["Status"]).values
y = df["Status"].values

# Compute Spearman correlation between rows (feature vectors)
rho_matrix, _ = spearmanr(X, axis=1)

# Separate correlations into intra-class and inter-class
classes = np.unique(y)
intra_class_corrs = []
inter_class_corrs = []

# Gather intra-class correlations
for c in classes:
    indices = np.where(y == c)[0]
    intra_corr_matrix = rho_matrix[np.ix_(indices, indices)]
    intra_class_corrs.extend(intra_corr_matrix[np.triu_indices(len(indices), k=1)])

# Gather inter-class correlations
for i in range(len(classes)):
    for j in range(i+1, len(classes)):
        indices_i = np.where(y == classes[i])[0]
        indices_j = np.where(y == classes[j])[0]
        inter_corr_matrix = rho_matrix[np.ix_(indices_i, indices_j)]
        inter_class_corrs.extend(inter_corr_matrix.ravel())

# Visualization of correlations
plt.figure(figsize=(10, 5))

# Histogram visualization
plt.hist(intra_class_corrs, bins=30, alpha=0.6, label="Intra-class", density=True, color='blue')
plt.hist(inter_class_corrs, bins=30, alpha=0.6, label="Inter-class", density=True, color='orange')
plt.title("Spearman Correlation Distribution (Feature Vectors)")
plt.xlabel("Spearman Correlation")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()

# Boxplot visualization
plt.figure(figsize=(6, 4))
sns.boxplot(data=[intra_class_corrs, inter_class_corrs], palette=['blue', 'orange'])
plt.xticks([0, 1], ['Intra-class', 'Inter-class'])
plt.title("Boxplot of Spearman Correlation Distributions")
plt.ylabel("Spearman Correlation")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

# Load and clean data
df = pd.read_csv("extracted_features_with_prep.csv")
df = df.drop(columns=[col for col in df.columns if "Code_Number_6digits" in col])
df = df.drop(columns=["Filename"])
X = df.drop(columns=["Status"])
y = df["Status"].values
classes = np.unique(y)

# Loop over each feature
for feature in X.columns:
    feature_vals = X[feature].values

    # Compute Spearman correlation between samples (via ranks + Pearson)
    corr_matrix, _ = spearmanr(np.expand_dims(feature_vals, axis=1), axis=1)

    # Safeguard in case it returns a float (shouldn't happen here, but safe)
    if isinstance(corr_matrix, float):
        corr_matrix = np.ones((len(feature_vals), len(feature_vals)))

    intra_feature_corrs = []
    inter_feature_corrs = []

    for c in classes:
        indices = np.where(y == c)[0]
        intra = corr_matrix[np.ix_(indices, indices)]
        intra_feature_corrs.extend(intra[np.triu_indices(len(indices), k=1)])

    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            idx_i = np.where(y == classes[i])[0]
            idx_j = np.where(y == classes[j])[0]
            inter = corr_matrix[np.ix_(idx_i, idx_j)]
            inter_feature_corrs.extend(inter.ravel())

    # Plotting
    plt.figure(figsize=(8, 4))
    plt.hist(intra_feature_corrs, bins=25, alpha=0.6, label="Intra-class", density=True, color='green')
    plt.hist(inter_feature_corrs, bins=25, alpha=0.6, label="Inter-class", density=True, color='red')
    plt.title(f"Spearman Correlation Distribution for Feature '{feature}'")
    plt.xlabel("Spearman Correlation")
    plt.ylabel("Density")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
df = pd.read_csv("extracted_features_with_prep.csv")
df = df.drop(columns=[col for col in df.columns if "Code_Number_6digits" in col])
df = df.drop(columns=[col for col in df.columns if "Filename" in col])

# Separate features and target.
features = df.drop(columns=["Status"])
target = df["Status"]

# Split the data into separate DataFrames for each class.
df_class0 = df[df["Status"] == 0].drop(columns=["Status"])
df_class1 = df[df["Status"] == 1].drop(columns=["Status"])

# Compute the correlation matrix for each class.
corr_class0 = df_class0.corr(method='spearman')
corr_class1 = df_class1.corr(method='spearman')


# Plot the correlation matrix for Class 0
plt.figure(figsize=(20, 16))
sns.heatmap(corr_class0, annot=False, cmap="coolwarm")
plt.title("Correlation Matrix for Class 0", fontsize=20)
plt.tight_layout()
plt.show()

# Plot the correlation matrix for Class 1
plt.figure(figsize=(20, 16))
sns.heatmap(corr_class1, annot=False, cmap="coolwarm")
plt.title("Correlation Matrix for Class 1", fontsize=20)
plt.tight_layout()
plt.show()

# Compute the difference
diff_corr = corr_class0 - corr_class1

# Plot the difference in correlations
plt.figure(figsize=(20, 16))
sns.heatmap(diff_corr, annot=False, cmap="coolwarm")
plt.title("Difference in Correlation (Class 0 - Class 1)", fontsize=20)
plt.tight_layout()
plt.show()


In [ ]:
df = pd.read_csv('Features_With_Preprocessing_Normalized.csv')

plt.figure(figsize=(15, 10))
df.hist(bins=30, figsize=(20, 15), edgecolor='black')
plt.suptitle("Feature Distributions after Normalization", fontsize=16)
plt.show()


Prep for Visulaization

In [ ]:
with_prep_file = 'Features_With_Preprocessing_Normalized.csv'
output_combined_file = 'combined_all_features_interleaved.csv'

with_prep_df = pd.read_csv(with_prep_file)
with_prep_df = with_prep_df.drop( columns=[col for col in with_prep_df.columns if "Code_Number_6digits" in col])



# Ensure the 'Status' column is numeric
with_prep_df['Status'] = pd.to_numeric(with_prep_df['Status'], errors='coerce')

# Drop rows with NaN in 'Status' if any exist after conversion
with_prep_df = with_prep_df.dropna(subset=['Status'])

# Separate data based on `Status`
status_1_data = with_prep_df[with_prep_df['Status'] == 1].drop(columns=['Status'])
status_0_data = with_prep_df[with_prep_df['Status'] == 0].drop(columns=['Status'])

# Interleave all features between `Status 1` and `Status 0`
interleaved_columns = []
for col in with_prep_df.columns:
    if col not in ['Filename', 'Status']:  # Exclude non-feature columns
        if col in status_1_data.columns:
            interleaved_columns.append(f"{col}_status_1")
        if col in status_0_data.columns:
            interleaved_columns.append(f"{col}_status_0")

# Include 'Filename' at the start
interleaved_columns =  interleaved_columns

# Rename columns to reflect status
status_1_data = status_1_data.rename(columns={col: f"{col}_status_1" for col in status_1_data.columns if col != 'Filename'})
status_0_data = status_0_data.rename(columns={col: f"{col}_status_0" for col in status_0_data.columns if col != 'Filename'})

# Combine interleaved data
combined_data = pd.concat([status_1_data, status_0_data], axis=1)

# Rearrange columns in the interleaved order
combined_data = combined_data[interleaved_columns]

# Save the interleaved data to a CSV file
combined_data.to_csv(output_combined_file, index=False)

print(f"Data has been processed and saved to: {output_combined_file}")


In [ ]:
with_prep_file = 'extracted_features_with_prep.csv'
output_combined_file_raw = 'combined_all_features_interleaved_raw.csv'

# Read the data from the CSV file
with_prep_df = pd.read_csv(with_prep_file)
with_prep_df = with_prep_df.drop( columns=[col for col in with_prep_df.columns if "Code_Number_6digits" in col])

# Ensure the 'Status' column is numeric
with_prep_df['Status'] = pd.to_numeric(with_prep_df['Status'], errors='coerce')

# Drop rows with NaN in 'Status' if any exist after conversion
with_prep_df = with_prep_df.dropna(subset=['Status'])

# Separate data based on `Status`
status_1_data = with_prep_df[with_prep_df['Status'] == 1].drop(columns=['Status'])
status_0_data = with_prep_df[with_prep_df['Status'] == 0].drop(columns=['Status'])

# Interleave all features between `Status 1` and `Status 0`
interleaved_columns = []
for col in with_prep_df.columns:
    if col not in ['Filename', 'Status']:  # Exclude non-feature columns
        if col in status_1_data.columns:
            interleaved_columns.append(f"{col}_status_1")
        if col in status_0_data.columns:
            interleaved_columns.append(f"{col}_status_0")

# Include 'Filename' at the start
interleaved_columns = ['Filename'] + interleaved_columns

# Rename columns to reflect status
status_1_data = status_1_data.rename(columns={col: f"{col}_status_1" for col in status_1_data.columns if col != 'Filename'})
status_0_data = status_0_data.rename(columns={col: f"{col}_status_0" for col in status_0_data.columns if col != 'Filename'})

# Combine interleaved data
combined_data = pd.concat([status_1_data, status_0_data], axis=1)

# Rearrange columns in the interleaved order
combined_data = combined_data[interleaved_columns]

# Save the interleaved data to a CSV file
combined_data.to_csv(output_combined_file_raw, index=False)

print(f"Data has been processed and saved to: {output_combined_file_raw}")


Visualization for Normalized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
data = pd.read_csv(output_combined_file)

# Drop unnecessary columns
data_dropped = data.drop(
    columns=[col for col in data.columns if 'Filename' in col],
    errors='ignore'
)
data_dropped = data_dropped.drop(
    columns=['Code_Number_6digits_status_0', 'Code_Number_6digits_status_1'],
    errors='ignore'
)
data_dropped = data_dropped.replace({np.nan: None})

# Helper to produce nicer labels
def display_name(col):
    if col.endswith('_status_1'):
        return f"{col[:-len('_status_1')]} (pfps positive)"
    if col.endswith('_status_0'):
        return f"{col[:-len('_status_0')]} (pfps negative)"
    return col

# Visualize features pair-wise with Box Plot
def visualize_features_box_pair(pair):
    # map raw column names → display names
    label_map = {c: display_name(c) for c in pair}

    subset_df = (
        data_dropped[pair]
        .melt(var_name='Feature', value_name='Value')
        .dropna()
    )
    subset_df['Feature'] = subset_df['Feature'].map(label_map)

    plt.figure(figsize=(8, 6))
    sns.boxplot(data=subset_df, x='Feature', y='Value', palette='Set2')

    title = "Box Plot — " + " vs ".join(label_map[c] for c in pair)
    plt.title(title, fontsize=14)
    plt.xlabel("Feature", fontsize=12)
    plt.ylabel("Normalized Value", fontsize=12)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Visualize features pair-wise with Violin Plot
def visualize_features_violin_pair(pair):
    label_map = {c: display_name(c) for c in pair}

    subset_df = (
        data_dropped[pair]
        .melt(var_name='Feature', value_name='Value')
        .dropna()
    )
    subset_df['Feature'] = subset_df['Feature'].map(label_map)

    plt.figure(figsize=(8, 6))
    sns.violinplot(data=subset_df, x='Feature', y='Value', palette='Set2')

    title = "Violin Plot — " + " vs ".join(label_map[c] for c in pair)
    plt.title(title, fontsize=14)
    plt.xlabel("Feature", fontsize=12)
    plt.ylabel("Normalized Value", fontsize=12)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

chunk_size = 2
columns = data_dropped.columns

# Iterate through each pair separately
for i in range(0, len(columns), chunk_size):
    pair = list(columns[i:i + chunk_size])

    visualize_features_box_pair(pair)
    visualize_features_violin_pair(pair)


split data

In [ ]:
df = pd.read_csv('extracted_features_with_prep.csv')
print("Shape of extracted_features_with_prep:", df.shape)

# Define features and target (adjust 'Status' to your actual target column if different)
X = df.drop(columns=['Status', 'Filename'])
y = df['Status']

# --- Standard Scaler ---
standard_scaler = StandardScaler()
X_standard = standard_scaler.fit_transform(X)
standard_scaled_df = pd.DataFrame(X_standard, columns=X.columns)
standard_scaled_df['Status'] = y.values  # Append target
print("Shape of standard_scaled_df:", standard_scaled_df.shape)

# --- Robust Scaler ---
robust_scaler = RobustScaler()
X_robust = robust_scaler.fit_transform(X)
robust_scaled_df = pd.DataFrame(X_robust, columns=X.columns)
robust_scaled_df['Status'] = y.values
print("Shape of robust_scaled_df:", robust_scaled_df.shape)

# --- MinMax Scaler ---
minmax_scaler = MinMaxScaler()
X_minmax = minmax_scaler.fit_transform(X)
minmax_scaled_df = pd.DataFrame(X_minmax, columns=X.columns)
minmax_scaled_df['Status'] = y.values
print("Shape of minmax_scaled_df:", minmax_scaled_df.shape)


# Output the DataFrames
standard_scaled_df, robust_scaled_df, minmax_scaled_df


Norm Test after standardization

In [ ]:
def analyze_features(file_path, group_col="Group", group1="PFPS", group2="Non-PFPS", features=None, output_file=None):

    import pandas as pd
    import numpy as np
    from scipy.stats import shapiro, ttest_ind, mannwhitneyu
    import os

    # Load the dataset
    data = pd.read_csv(file_path)
    print(f"Analyzing file: {file_path}")
    print(data.head())

    if features is None:
        features = data.select_dtypes(include=[np.number]).columns.tolist()
        if group_col in features:
            features.remove(group_col)

    results = []

    for feature in features:

        if feature not in data.columns:
            continue

        print(f"\nAnalyzing Feature: {feature}")

        # Compute descriptive statistics for each group
        mean_feature = data.groupby(group_col)[feature].mean()
        median_feature = data.groupby(group_col)[feature].median()
        mode_feature = data.groupby(group_col)[feature].agg(lambda x: x.mode()[0] if not x.mode().empty else None)

        # Extract the values for group1 and group2, dropping NaN values
        group1_vals = data[data[group_col] == group1][feature].dropna()
        group2_vals = data[data[group_col] == group2][feature].dropna()

        # Perform Shapiro-Wilk test on group1 values (to assess normality)
        try:
            stat_shapiro, p_shapiro = shapiro(group1_vals)
        except Exception as e:
            print(f"Shapiro test error for {feature}: {e}")
            p_shapiro = np.nan

        # Decide which test to use based on normality of group1
        if p_shapiro >= 0.05:
            test_used = "t-Test"
            try:
                t_stat, p_test = ttest_ind(group1_vals, group2_vals, equal_var=False)
            except Exception as e:
                print(f"t-Test error for {feature}: {e}")
                t_stat, p_test = np.nan, np.nan
        else:
            test_used = "Mann-Whitney U Test"
            try:
                u_stat, p_test = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
            except Exception as e:
                print(f"Mann-Whitney test error for {feature}: {e}")
                u_stat, p_test = np.nan, np.nan

        # Record the results for this feature
        results.append({
            "Feature": feature,
            f"{group1}_Mean": mean_feature.get(group1, np.nan),
            f"{group2}_Mean": mean_feature.get(group2, np.nan),
            f"{group1}_Median": median_feature.get(group1, np.nan),
            f"{group2}_Median": median_feature.get(group2, np.nan),
            f"{group1}_Mode": mode_feature.get(group1, np.nan),
            f"{group2}_Mode": mode_feature.get(group2, np.nan),
            "Shapiro_PValue": p_shapiro,
            "Test_Used": test_used,
            "Test_Statistic": t_stat if test_used=="t-Test" else u_stat,
            "Test_PValue": p_test
        })

    # Convert results into a DataFrame and save to CSV
    results_df = pd.DataFrame(results)

    if output_file is None:
        base = os.path.splitext(os.path.basename(file_path))[0]
        output_file = f"{base}_feature_analysis.csv"

    results_df.to_csv(output_file, index=False)
    print(f"\nAnalysis complete. Results saved to '{output_file}'.")
    return results_df




# Save the normalized datasets to CSV files
standard_scaled_df.to_csv("Standard_norm.csv", index=False)
robust_scaled_df.to_csv("Robust_norm.csv", index=False)
minmax_scaled_df.to_csv("MinMax_norm.csv", index=False)

analyze_features("Standard_norm.csv", group_col="Status", group1=0, group2=1)
analyze_features("Robust_norm.csv", group_col="Status", group1=0, group2=1)
analyze_features("MinMax_norm.csv", group_col="Status", group1=0, group2=1)


In [ ]:
df = pd.read_csv("Standard_norm.csv")
df.columns

Visualize after normalization

In [ ]:
# Define the features to plot against "Status"
features = [
      'Mean', 'Std', 'Variance', 'Skewness', 'Kurtosis',
       'Dissimilarity_0', 'Dissimilarity_45', 'Dissimilarity_90',
       'Dissimilarity_135', 'Correlation_0', 'Correlation_45',
       'Correlation_90', 'Correlation_135', 'Homogeneity_0', 'Homogeneity_45',
       'Homogeneity_90', 'Homogeneity_135', 'Energy_0', 'Energy_45',
       'Energy_90', 'Energy_135', 'Contrast_0', 'Contrast_45', 'Contrast_90',
       'Contrast_135', 'RunLengthNonUniformity', 'ShortRunEmphasis',
       'LongRunEmphasis', 'GrayLevelNonUniformity', 'RunPercentage',
       'RunEntropy', 'Area', 'Eccentricity', 'Perimeter', 'Centroid_X',
       'Centroid_Y', 'Hu0',	'Hu1',	'Hu2',	'Hu3',	'Hu4',	'Hu5',	'Hu6', 'Age',
       'Gender', 'Active Playing Years', 'Height', 'Weight',
       'Leg Length', 'Q-Angle', 'Active Knee'
            ]

# Function to plot histograms for scaled datasets
def plot_histograms(scaled_df, title):
    plt.figure(figsize=(15, 10))
    scaled_df.hist(bins=30, figsize=(20, 15), edgecolor='black')
    plt.suptitle(title, fontsize=16)
    plt.show()

def plot_boxplots(scaled_df, title):
    plt.figure(figsize=(20, 10))
    sns.boxplot(data=scaled_df, whis=1.5)  # 'whis' controls whisker length
    plt.xticks(rotation=90)  # Rotate feature labels for readability
    plt.title(title, fontsize=16)
    plt.show()

# Function to create scatter plots for all features in a single figure
def plot_scatter_grid(scaled_df, title):
    num_features = len(features)
    num_cols = 5  # Number of columns for the grid
    num_rows = (num_features // num_cols) + (num_features % num_cols > 0)  # Calculate rows

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 15))
    fig.suptitle(title, fontsize=18)
    axes = axes.flatten()  # Flatten the axes array for easy iteration

    for i, feature in enumerate(features):
        # Use the 'Status' column from the DataFrame directly
        sns.scatterplot(ax=axes[i], data=scaled_df, x=feature, y='Status', alpha=0.5)
        axes[i].set_title(feature)
        axes[i].set_xlabel("")
        axes[i].set_ylabel("Status")

    # Hide any empty subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit title
    plt.show()

# Visualize distributions after applying different scalers
plot_histograms(standard_scaled_df, "Feature Distributions After Standard Scaling")
plot_histograms(robust_scaled_df, "Feature Distributions After Robust Scaling")
plot_histograms(minmax_scaled_df, "Feature Distributions After MinMax Scaling")

# Visualize distributions using boxplots for each scaler
plot_boxplots(standard_scaled_df, "Feature Distributions After Standard Scaling (Boxplot)")
plot_boxplots(robust_scaled_df, "Feature Distributions After Robust Scaling (Boxplot)")
plot_boxplots(minmax_scaled_df, "Feature Distributions After MinMax Scaling (Boxplot)")

# Generate scatter plots for each scaling method
plot_scatter_grid(standard_scaled_df, "Standard Scaled Data - Scatter Plots")
plot_scatter_grid(robust_scaled_df, "Robust Scaled Data - Scatter Plots")
plot_scatter_grid(minmax_scaled_df, "MinMax Scaled Data - Scatter Plots")


Feature Selection

sources:
1) source for feature ranking using random forest feature importance and rfe for features selection: https://www.mdpi.com/2224-2708/12/5/67?utm



In [ ]:
# Define the allowed features list from stat test
allowed_features = [

    "Mean",
    "Skewness",
    "Kurtosis",
    "Dissimilarity_0",
    "Dissimilarity_45",
    "Dissimilarity_90",
    "Dissimilarity_135",
    "Homogeneity_0",
    "Homogeneity_45",
    "Homogeneity_90",
    "Homogeneity_135",
    "Energy_0",
    "Energy_45",
    "Energy_90",
    "Energy_135",
    "Contrast_45",
    "Contrast_90",
    "RunLengthNonUniformity",
    "LongRunEmphasis",
    "GrayLevelNonUniformity",
    "RunPercentage",
    "RunEntropy",
    "Area",
    "Eccentricity",
    "Perimeter",
    "Hu3",
    "Hu4",
    "Hu6",

    "Gender",
    "Active Playing Years",
    "Height",
    "Q-Angle"
]




# Load the dataset
df = pd.read_csv('extracted_features_with_prep.csv')
print("Shape of extracted_features_with_prep:", df.shape)
df = df[allowed_features + ['Status', 'Filename'] ]

# Define features and target and non feature
X = df.drop(columns=['Status', 'Filename'])
y = df['Status']

# Use StratifiedKFold to maintain class balance
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store results from each combination of scaler and feature selection method
results = {}

# Create scaled datasets
scaled_data = {}

standard_scaler = StandardScaler()
robust_scaler = RobustScaler()
minmax_scaler = MinMaxScaler()

X_standard = standard_scaler.fit_transform(X)
X_robust = robust_scaler.fit_transform(X)
X_minmax = minmax_scaler.fit_transform(X)

df_standard = pd.DataFrame(X_standard, columns=X.columns)
df_standard['Status'] = y.values
df_robust = pd.DataFrame(X_robust, columns=X.columns)
df_robust['Status'] = y.values
df_minmax = pd.DataFrame(X_minmax, columns=X.columns)
df_minmax['Status'] = y.values

scaled_data["Standard"] = df_standard
scaled_data["Robust"] = df_robust
scaled_data["MinMax"] = df_minmax

# Loop over each scaler
for scaler_name, df_scaled in scaled_data.items():
    print(f"\n===== Processing {scaler_name} Scaled Data =====")
    # Start with the full set of features
    X_scaled = df_scaled.drop(columns=['Status'])
    y_scaled = df_scaled['Status']


    # Allowed Features Dropping Method
    X_allowed = X_scaled[[col for col in X_scaled.columns if col in allowed_features]]
    print(f"{scaler_name} features after allowed_features filtering: {list(X_allowed.columns)}")
    allowed_df = X_allowed.copy()
    allowed_df['Status'] = y_scaled.values
    key_allowed = f"{scaler_name}_Allowed"
    results[key_allowed] = allowed_df.copy()
    print(f"{scaler_name}_Allowed dataset stored with {allowed_df.shape[1]-1} features (excluding 'Status').")

    # RFECV Feature Selection with Logistic Regression
    print(f"\n=== {scaler_name} Scaling: RFECV Feature Selection (Logistic Regression) with Undersampling ===")
    pipeline_rfecv_lr = ImbPipeline([
        ('undersample', RandomUnderSampler(random_state=42)),
        ('feature_selection', RFECV(estimator=LogisticRegression(solver='liblinear', penalty='l1', max_iter=1000),
                                    cv=5, scoring='accuracy')),
        ('clf', LogisticRegression(solver='liblinear', penalty='l1', max_iter=1000))
    ])
    param_grid_lr = {
        'feature_selection__min_features_to_select': [10, 20],
        'clf__C': [0.1, 1, 10, 100]
    }
    nested_cv_lr = GridSearchCV(pipeline_rfecv_lr, param_grid_lr,
                             cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                             scoring='accuracy', n_jobs=-1)
    nested_cv_lr.fit(X_scaled, y_scaled)

    best_fs_lr = nested_cv_lr.best_estimator_.named_steps['feature_selection']
    selected_features_lr = X_scaled.columns[best_fs_lr.support_]
    if len(selected_features_lr) == 0:
        print(f"No features were selected for {scaler_name} using L1 Logistic Regression. Consider adjusting C or using L2 penalty.")
    else:
        X_rfecv_lr_selected = X_scaled[selected_features_lr]
        rfecv_lr_df = X_rfecv_lr_selected.copy()
        rfecv_lr_df['Status'] = y_scaled.values
        key_rfecv = f"{scaler_name}_RFECV_LR"
        results[key_rfecv] = rfecv_lr_df

        # Visualization for RFECV l1 Logistic Regression
        clf_model_lr = nested_cv_lr.best_estimator_.named_steps['clf']
        coefs = clf_model_lr.coef_[0]
        coef_df = pd.DataFrame({'Feature': selected_features_lr, 'Coefficient': coefs})
        if not coef_df.empty:
            plt.figure(figsize=(8, 6))
            sns.barplot(data=coef_df.sort_values(by='Coefficient', key=abs, ascending=False),
                        x='Coefficient', y='Feature', palette='coolwarm')
            plt.title(f"{scaler_name} Scaling: RFECV (LR) Selected Feature Coefficients")
            plt.tight_layout()
            plt.show()
        else:
            print(f"No features selected for visualization in {scaler_name} RFECV LR.")


    # === PCA Feature Selection with Full Cumulative‑Loading Debugging ===
    print(f"\n=== {scaler_name} Scaling: PCA Feature Selection ===")

    # Full PCA to inspect all PC
    pca_full = PCA()
    pca_full.fit(X_scaled)
    explained = pca_full.explained_variance_ratio_
    cum_var_full = np.cumsum(explained)

    # print each PC’s ratio & cumulative for checking
    print("PC   VarRatio   CumulativeVar")
    for i, (ev, cv) in enumerate(zip(explained, cum_var_full), start=1):
        print(f"PC{i:02d}  {ev:8.4f}     {cv:8.4f}")

    # Plot full cumulative explained variance
    plt.figure(figsize=(8,5))
    plt.plot(np.arange(1, len(cum_var_full)+1), cum_var_full, marker='o')
    plt.axhline(0.90, color='gray', linestyle='--', label='90% threshold')
    plt.xlabel('Principal Component')
    plt.ylabel('Cumulative Explained Variance')
    plt.title(f"{scaler_name} Scaling: PCA Cumulative Explained Variance (All PCs)")
    plt.xticks(np.arange(1, len(cum_var_full)+1))
    plt.legend()
    plt.tight_layout()
    plt.show()

    # PCA up to 90% to get the number of PCs
    pca90 = PCA(n_components=0.90)
    pca90.fit(X_scaled)
    n_pcs_90 = pca90.n_components_
    print(f"Number of PCs to reach ≥90% variance: {n_pcs_90}")

    # Compute loading “mass” over those top PCs
    loadings = pca90.components_                          # shape = (n_pcs_90, n_features)
    contribs = np.sum(np.abs(loadings), axis=0)
    total_loading = contribs.sum()

    # Build a DataFrame of feature → contrib → cum_pct
    contrib_df = (
        pd.DataFrame({
            'feature': X_scaled.columns,
            'contrib': contribs
        })
        .sort_values('contrib', ascending=False)
        .reset_index(drop=True)
    )
    contrib_df['cum_pct'] = contrib_df['contrib'].cumsum() / total_loading

    # Debug print: all features with their cumulative loading percentages
    print("\nFeature                     Contrib    CumPct")
    for _, row in contrib_df.iterrows():
        print(f"{row.feature:25s}  {row.contrib:8.4f}  {row.cum_pct:8.4f}")

    # Select features whose cum_pct ≤ 0.90
    selected_features_pca = contrib_df.loc[contrib_df['cum_pct'] <= 0.90, 'feature'].tolist()
    print(f"\nSelected {len(selected_features_pca)} features covering ≤90% loading mass:")
    print(selected_features_pca)

    # Store both the full contrib_df and the reduced dataset
    results[f"{scaler_name}_PCA_Loadings"] = contrib_df.copy()
    X_pca_sel = X_scaled[selected_features_pca]
    pca_df = pd.DataFrame(X_pca_sel, columns=selected_features_pca)
    pca_df['Status'] = y_scaled.values
    results[f"{scaler_name}_PCA"] = pca_df

    # Plot Loading Masses per Feature
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(data=contrib_df, x='contrib', y='feature', palette='viridis')
    ax.set_xlabel('Loading Mass')
    ax.set_ylabel('Feature')
    ax.set_title(f"{scaler_name} Scaling: PCA Loading Mass per Feature")

    """
    # Annotate each bar with its numeric value
    for idx, row in contrib_df.iterrows():
        ax.text(
            row.contrib + 0.02,
            idx,
            f"{row.contrib:.2f}",
            va='center'
        )
    """
    plt.tight_layout()
    plt.show()


    # Mutual Information Feature Selection
    print(f"\n=== {scaler_name} Scaling: Mutual Information Feature Selection ===")
    from sklearn.feature_selection import mutual_info_classif
    mi_scores = mutual_info_classif(X_scaled, y_scaled, random_state=42)
    mi_df = pd.DataFrame({'Feature': X_scaled.columns, 'MI_Score': mi_scores})
    mi_df = mi_df.sort_values(by='MI_Score', ascending=False)

    # Select features with MI score above the median score
    median_mi = mi_df['MI_Score'].median()
    selected_features_mi = mi_df[mi_df['MI_Score'] >= median_mi]['Feature'].tolist()
    X_mi_selected = X_scaled[selected_features_mi]
    mi_df_final = X_mi_selected.copy()
    mi_df_final['Status'] = y_scaled.values
    key_mi = f"{scaler_name}_MutualInfo"
    results[key_mi] = mi_df_final

    # Visualization for Mutual Information: Feature Scores
    plt.figure(figsize=(8,6))
    sns.barplot(data=mi_df, x='MI_Score', y='Feature', palette='Blues_d')
    plt.title(f"{scaler_name} Scaling: Mutual Information Feature Scores")
    plt.tight_layout()
    plt.show()

# Save Full Feature Set Without Any Reduction (uses full scaled data)

for scaler_name, df_scaled in scaled_data.items():
    key_full = f"{scaler_name}_Full"
    results[key_full] = df_scaled.copy()
    print(f"{scaler_name}_Full dataset stored with {df_scaled.shape[1]-1} features (excluding 'Status').")

# Save all results using a loop
for key, result_df in results.items():
    filename = f"final_{key}.csv"
    result_df.to_csv(filename, index=False)
    print(f"{key} complete. Saved as {filename}")

print("\n=== All feature selection methods on scaled data, including PCA and Mutual Information, completed successfully. ===")


In [ ]:
lol = "final_MinMax_PCA.csv"
lol2 = pd.read_csv(lol)
lol2.columns


In [ ]:
lol = pd.read_csv("extracted_features_with_prep.csv")

lol.columns

Norm test after reduction

In [ ]:
final_files = [
    "final_MinMax_Allowed.csv",
    "final_MinMax_RFECV_LR.csv",
    "final_MinMax_PCA.csv",
    "final_MinMax_MutualInfo.csv",
    "final_MinMax_Full.csv",

    "final_Standard_Allowed.csv",
    "final_Standard_RFECV_LR.csv",
    "final_Standard_PCA.csv",
    "final_Standard_MutualInfo.csv",
    "final_Standard_Full.csv",

    "final_Robust_Allowed.csv",
    "final_Robust_RFECV_LR.csv",
    "final_Robust_PCA.csv",
    "final_Robust_MutualInfo.csv",
    "final_Robust_Full.csv"
]

# Loop over each file and analyze it, if it contains the 'Status' column
for file in final_files:
    try:
        df = pd.read_csv(file)
        print(f"\n\033[1;34m=== {file} ===\033[0m")
        print(f"Shape: {df.shape}")
        display(df.head())
        if "Status" not in df.columns:
            print(f"Skipping file {file} because it does not contain 'Status'.")
            continue
        # Call the analyze_features function (ensure it is defined or imported).
        # Adjust group1 and group2 values as needed.
        analyze_features(file, group_col="Status", group1=0, group2=1)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Download all final output CSV files.
analysis_files = glob.glob("final_*_feature_analysis.csv") + glob.glob("final_*_feature_ranking.csv")
all_files = final_files + analysis_files

for file in all_files:
    if os.path.isfile(file):
        print("Downloading:", file)
        files.download(file)
    else:
        print(f"File not found: {file}")

print("\n=== Download of all final output files completed. ===")


Validate

In [ ]:
lol = "final_Standard_Allowed.csv"
lol2 = pd.read_csv(lol)
lol2.columns

with MSE, with Grid Search F1 score

In [ ]:
# Suppress warnings for undefined metrics and deprecated messages.
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", message="`use_label_encoder` is deprecated")


# Learning Curve Function using Training Loss (Log Loss)
def plot_learning_curve(estimator, title, X, y, cv, scoring):
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1
    )

    if scoring == 'neg_log_loss':
        train_loss = -np.mean(train_scores, axis=1)
        val_loss = -np.mean(val_scores, axis=1)
    else:
        train_loss = 1 - np.mean(train_scores, axis=1)
        val_loss = 1 - np.mean(val_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_loss, 'o-', color="r", label="Training Loss")
    plt.plot(train_sizes, val_loss, 'o-', color="g", label="Validation Loss")
    plt.xlabel("Number of Training Samples")
    plt.ylabel("Log Loss")
    plt.title(title)
    plt.legend(loc="best")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Updated Function for Training vs. Testing Error using Log Loss
def plot_train_test_error(model, X_train, y_train, X_test, y_test, model_name, method_name, scoring="neg_log_loss"):

    try:
        y_train_pred_prob = model.predict_proba(X_train)
        y_test_pred_prob = model.predict_proba(X_test)
    except Exception as e:
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        y_train_pred_prob = np.vstack([1 - y_train_pred, y_train_pred]).T
        y_test_pred_prob = np.vstack([1 - y_test_pred, y_test_pred]).T

    train_loss = log_loss(y_train, y_train_pred_prob)
    test_loss = log_loss(y_test, y_test_pred_prob)
    metric_label = "Log Loss"

    loss_difference = test_loss - train_loss
    loss_ratio = test_loss / train_loss if train_loss != 0 else float('inf')

    print(f"{model_name} ({method_name}):")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Testing Loss:  {test_loss:.4f}")
    print(f"  Loss Difference (Test - Train): {loss_difference:.4f}")
    print(f"  Loss Ratio (Test / Train): {loss_ratio:.2f}")

    if loss_ratio > 1.2:
        print("  Warning: Test Loss is significantly higher than Training Loss. Possible overfitting detected.")
    else:
        print("  Training and Testing Loss are similar. Overfitting is less likely.")

    plt.figure(figsize=(5,4))
    plt.bar(["Train Loss", "Test Loss"], [train_loss, test_loss], color=["#1f77b4", "#ff7f0e"])
    plt.title(f"Training vs. Testing Loss\n{model_name} ({method_name})")
    plt.ylabel(metric_label)
    plt.grid(axis='y', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Precision-Recall Plotting Function
def plot_precision_recall_and_threshold(y_true, y_scores, model_name, method_name):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    # Compute F1 scores for thresholds (thresholds has one less element)
    f1_scores = 2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-10)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    print(f"Optimal threshold for {model_name} ({method_name}) based on max F1 score: {best_threshold:.2f}")
    avg_precision = average_precision_score(y_true, y_scores)

    plt.figure(figsize=(8,6))
    plt.plot(recall, precision, label=f'AP = {avg_precision:.2f}')
    plt.scatter(recall[best_idx], precision[best_idx], color='red',
                label=f'Optimal (F1 = {f1_scores[best_idx]:.2f})')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve: {model_name} ({method_name})\n(Maximizing F1 Score)")
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.show()
    return best_threshold


# Load Feature Selection Outputs
print("=== Step 1: Load Feature Selection Outputs ===")
scalers = ["Standard", "MinMax", "Robust"]
methods = ["Allowed", "RFECV_LR", "PCA", "MutualInfo", "Full"]

feature_selection_methods = {}
for scaler in scalers:
    for method in methods:
        key = f"{scaler}_{method}"
        filename = f"final_{key}.csv"
        try:
            df = pd.read_csv(filename)
            feature_selection_methods[key] = df
            print(f"Loaded {filename}")
        except Exception as e:
            print(f"Error loading {filename}: {e}")


# Define Models for Testing
def define_models():
    return {
        "SVM": SVC(kernel='linear', probability=True, random_state=42, max_iter=5000),
        "SVM_RBF": SVC(kernel='rbf', probability=True, random_state=42, max_iter=5000),   # Added SVM RBF
        "SVM_Poly": SVC(kernel='poly', probability=True, random_state=42, max_iter=5000),  # Added SVM Pol
        "L1LogisticRegression": LogisticRegression(penalty='l1', solver='liblinear',
                                                   class_weight='balanced', random_state=42, max_iter=1000),
        "LDA": LinearDiscriminantAnalysis()
    }

# Set the grid search scoring metric to maximize
grid_search_scoring = "f1"


# Define function to train and evaluate models with Partial Undersampling + Class Weighting and visualizations
def train_and_evaluate_models(X_df, method_name):
    print(f"\n\033[1;34m=== TESTING FEATURE SELECTION METHOD: {method_name} ===\033[0m")

    # Partial Undersampling Before Splitting with Class Weighting
    df_combined = X_df.copy()
    counts = df_combined['Status'].value_counts()
    if len(counts) == 2:
        minority_class = counts.idxmin()
        majority_class = counts.idxmax()
        minority_count = counts.min()
        desired_ratio = 2
        df_minority = df_combined[df_combined['Status'] == minority_class]
        df_majority = df_combined[df_combined['Status'] == majority_class]
        sample_size = min(len(df_majority), desired_ratio * minority_count)
        df_majority_sampled = df_majority.sample(sample_size, random_state=42)
        df_undersampled = pd.concat([df_minority, df_majority_sampled])
    else:
        df_undersampled = df_combined
    print("Data shape after partial undersampling:", df_undersampled.shape)

    # Separate features and target
    X_features = df_undersampled.drop(columns=['Status'])
    y_target = df_undersampled['Status']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_target, test_size=0.2, random_state=42, stratify=y_target
    )
    print("Training set shape:", X_train.shape)
    print("Testing set shape:", X_test.shape)

    models = define_models()
    results_local = {}
    optimized_local = {}

    def plot_confusion_matrix(cm, title):
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title(title)
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

    def get_scoring(estimator):

        return 'neg_log_loss'

    for name, model in models.items():
        print(f"\n=== Training and Evaluating Model: {name} ({method_name}) ===")
        scoring_metric = get_scoring(model)

        # Learning Curve for the Initial (Pre-Optimized) Model
        print(f"\n--- Learning Curve for initial {name} model ({method_name}) ---")
        plot_learning_curve(
            model,
            f"Initial {name} Learning Curve ({method_name})",
            X_train, y_train,
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring=scoring_metric
        )

        # Pre-Optimized Train vs. Test Error (Using Log Loss)
        print(f"\n--- Pre-Optimized {name} Train vs. Test Error ({method_name}) ---")
        try:
            model.fit(X_train, y_train)
            plot_train_test_error(
                model, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PreOptim", method_name=method_name,
                scoring=scoring_metric
            )
        except Exception as e:
            print(f"Error fitting pre-optimized {name}: {e}")

        # PRE-OPTIMIZED KFold Cross-Validation Accuracy
        print(f"\n--- Pre-Optimized KFold CV Accuracy for {name} ({method_name}) ---")
        cv_pre = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        pre_fold_accuracies = []
        for fold, (train_idx, val_idx) in enumerate(cv_pre.split(X_train, y_train), 1):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            model.fit(X_fold_train, y_fold_train)
            acc = model.score(X_fold_val, y_fold_val)
            pre_fold_accuracies.append(acc)
            print(f"{name} (Pre-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
        mean_pre_acc = np.mean(pre_fold_accuracies)
        print(f"{name} (Pre-Optimized) - Mean CV Accuracy: {mean_pre_acc:.4f}")
        plt.figure(figsize=(6,4))
        plt.bar(range(1, len(pre_fold_accuracies)+1), pre_fold_accuracies, color="#2ca02c")
        plt.title(f"Pre-Optimized CV Accuracy per Fold for {name}")
        plt.xlabel("Fold Number")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()

        # Hyperparameter Tuning / Grid Search using f1 score
        try:
            if name == "L1LogisticRegression":
                param_grid = {"clf__class_weight": [{0: 1, 1: w} for w in [2, 3, 4, 5, 6]]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=grid_search_scoring,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name in ["SVM", "SVM_RBF"]:
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100]}
                if name == "SVM_RBF":
                    param_grid["clf__gamma"] = ['scale', 'auto']
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=grid_search_scoring,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "SVM_Poly":
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100],
                              "clf__degree": [2, 3, 4]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=grid_search_scoring,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "LDA":
                param_grid = {
                    "clf__solver": ["lsqr", "eigen"],
                    "clf__shrinkage": [0.3, 0.5, 0.7]
                }
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=grid_search_scoring,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                    n_jobs=-1, error_score='raise'
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            else:
                model_optimized = model

            optimized_local[name] = model_optimized

            # Learning Curve for the Optimized Model
            print(f"\n--- Learning Curve for optimized {name} model ({method_name}) ---")
            plot_learning_curve(
                model_optimized,
                f"Optimized {name} Learning Curve ({method_name})",
                X_train, y_train,
                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                scoring=scoring_metric
            )

            # Post-Optimized Train vs. Test Error (Using Log Loss)
            print(f"\n--- Post-Optimized {name} Train vs. Test Loss ({method_name}) ---")
            model_optimized.fit(X_train, y_train)
            plot_train_test_error(
                model_optimized, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PostOptim", method_name=method_name,
                scoring=scoring_metric
            )

            # Post-Optimized KFold Cross-Validation Accuracy Visualization
            cv_post = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            post_fold_accuracies = []
            for fold, (train_idx, val_idx) in enumerate(cv_post.split(X_train, y_train), 1):
                X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                model_optimized.fit(X_fold_train, y_fold_train)
                acc = model_optimized.score(X_fold_val, y_fold_val)
                post_fold_accuracies.append(acc)
                print(f"{name} (Post-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
            mean_post_acc = np.mean(post_fold_accuracies)
            print(f"{name} (Post-Optimized) - Mean CV Accuracy: {mean_post_acc:.4f}")
            plt.figure(figsize=(6,4))
            plt.bar(range(1, len(post_fold_accuracies)+1), post_fold_accuracies, color="#2ca02c")
            plt.title(f"Post-Optimized CV Accuracy per Fold for {name}")
            plt.xlabel("Fold Number")
            plt.ylabel("Accuracy")
            plt.ylim(0, 1)
            plt.tight_layout()
            plt.show()

            # Unadjusted and Adjusted Confusion Matrices
            if hasattr(model_optimized, "predict_proba"):
                # Compute and display unadjusted predictions and confusion matrix.
                y_pred = model_optimized.predict(X_test)
                cm_unadjusted = confusion_matrix(y_test, y_pred)
                report_unadjusted = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                print(f"\nUnadjusted Classification Report for {name} ({method_name}):")
                print(classification_report(y_test, y_pred, zero_division=0))
                plot_confusion_matrix(cm_unadjusted, f"Unadjusted Confusion Matrix: {name} ({method_name})")

                # Now apply threshold adjustment using precision-recall.
                y_scores = model_optimized.predict_proba(X_test)[:, 1]
                best_threshold = plot_precision_recall_and_threshold(
                    y_test, y_scores, model_name=name, method_name=method_name
                )
                y_pred_thresh = (y_scores >= best_threshold).astype(int)
                report_adjusted = classification_report(y_test, y_pred_thresh, zero_division=0, output_dict=True)
                print(f"\nAdjusted Classification Report at threshold {best_threshold:.2f} for {name} ({method_name}):")
                print(classification_report(y_test, y_pred_thresh, zero_division=0))
                cm_adjusted = confusion_matrix(y_test, y_pred_thresh)
                plot_confusion_matrix(cm_adjusted, f"Final Adjusted Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix_unadjusted": cm_unadjusted,
                    "classification_report_unadjusted": report_unadjusted,
                    "confusion_matrix_adjusted": cm_adjusted,
                    "classification_report_adjusted": report_adjusted,
                    "FeatureSelectionMethod": method_name
                }
            else:
                y_pred = model_optimized.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                report = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                plot_confusion_matrix(cm, f"Final Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix": cm,
                    "classification_report_unadjusted": report,
                    "FeatureSelectionMethod": method_name
                }

        except Exception as e:
            print(f"Error evaluating {name} on {method_name}: {e}")
            results_local[name] = {"error": str(e), "FeatureSelectionMethod": method_name}

    return {"performance": results_local, "optimized_models": optimized_local}


# Run Evaluations on Each Feature Selection Method
evaluation_results = {}
for method_key, df in feature_selection_methods.items():
    evaluation_results[method_key] = train_and_evaluate_models(df, method_key)

print("\n=== Model Evaluation Completed for All Feature Selection Methods ===")

# Save evaluation results
joblib.dump(evaluation_results, 'evaluation_results.pkl')
print("Evaluation results saved to 'evaluation_results.pkl'")

optimized_models = {}
for method_key, result in evaluation_results.items():
    optimized_models[method_key] = result.get("optimized_models", {})
joblib.dump(optimized_models, 'optimized_models.pkl')
print("Optimized models saved to 'optimized_models.pkl'")


In [ ]:
lol = "final_Standard_RFECV_LR.csv"
lol2 = pd.read_csv(lol)
lol2.columns

Grid search multiply

In [ ]:
# Suppress warnings for undefined metrics and deprecated messages.
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", message="`use_label_encoder` is deprecated")


# Product of Precision & Recall
def precision_recall_product(y_true, y_pred):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return p * r

product_pr_scorer = make_scorer(precision_recall_product, greater_is_better=True)

# Learning Curve Function using Training Loss (Log Loss)
def plot_learning_curve(estimator, title, X, y, cv, scoring):
    """
    Plots the learning curve and returns training sizes, training loss, and validation loss.
    """
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1
    )

    # Convert negative log loss to positive loss if needed.
    if scoring == 'neg_log_loss':
        train_loss = -np.mean(train_scores, axis=1)
        val_loss = -np.mean(val_scores, axis=1)
    else:
        train_loss = 1 - np.mean(train_scores, axis=1)
        val_loss = 1 - np.mean(val_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_loss, 'o-', color="r", label="Training Loss")
    plt.plot(train_sizes, val_loss, 'o-', color="g", label="Validation Loss")
    plt.xlabel("Number of Training Samples")
    plt.ylabel("Log Loss")
    plt.title(title)
    plt.legend(loc="best")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return train_sizes, train_loss, val_loss

# flatten nested dictionaries for CSV storage
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


# Updated Train-vs-Test Error Function Using Log Loss
def plot_train_test_error(model, X_train, y_train, X_test, y_test, model_name, method_name, scoring="neg_log_loss"):
    """
    Plots a bar chart comparing training and testing log loss,
    and prints the loss difference and ratio.
    """
    try:
        y_train_pred_prob = model.predict_proba(X_train)
        y_test_pred_prob = model.predict_proba(X_test)
    except Exception as e:
        print("Model does not support predict_proba, falling back to dummy conversion.")
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        y_train_pred_prob = np.vstack([1 - y_train_pred, y_train_pred]).T
        y_test_pred_prob = np.vstack([1 - y_test_pred, y_test_pred]).T

    train_loss = log_loss(y_train, y_train_pred_prob)
    test_loss = log_loss(y_test, y_test_pred_prob)
    metric_label = "Log Loss"

    loss_difference = test_loss - train_loss
    loss_ratio = test_loss / train_loss if train_loss != 0 else float('inf')

    print(f"{model_name} ({method_name}):")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Testing Loss:  {test_loss:.4f}")
    print(f"  Loss Difference (Test - Train): {loss_difference:.4f}")
    print(f"  Loss Ratio (Test / Train): {loss_ratio:.2f}")

    if loss_ratio > 1.2:
        print("  Warning: Test Loss is significantly higher than Training Loss. Possible overfitting detected.")
    else:
        print("  Training and Testing Loss are similar. Overfitting is less likely.")

    plt.figure(figsize=(5,4))
    plt.bar(["Train Loss", "Test Loss"], [train_loss, test_loss], color=["#1f77b4", "#ff7f0e"])
    plt.title(f"Training vs. Testing Loss\n{model_name} ({method_name})")
    plt.ylabel(metric_label)
    plt.grid(axis='y', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Precision-Recall Plotting Function Using Product Metric
def plot_precision_recall_and_threshold(y_true, y_scores, model_name, method_name):
    """
    Plots the Precision-Recall curve and finds the threshold where the product
    of precision and recall is maximized.
    """
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    # Calculate product of precision and recall for each threshold (exclude last point)
    prod_scores = precision[:-1] * recall[:-1]
    best_idx = np.argmax(prod_scores)
    best_threshold = thresholds[best_idx]
    print(f"Optimal threshold for {model_name} ({method_name}) based on max (Precision x Recall): {best_threshold:.2f}")
    avg_precision = average_precision_score(y_true, y_scores)

    plt.figure(figsize=(8,6))
    plt.plot(recall, precision, label=f'AP = {avg_precision:.2f}')
    plt.scatter(recall[best_idx], precision[best_idx], color='red',
                label=f'Optimal (Prod = {prod_scores[best_idx]:.2f})')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve: {model_name} ({method_name})\n(Maximizing Product)")
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.show()
    return best_threshold


# Load Feature Selection Outputs
print("=== Step 1: Load Feature Selection Outputs ===")

scalers = ["Standard", "MinMax", "Robust"]
methods = ["Allowed", "RFECV_LR", "PCA", "MutualInfo", "Full"]

feature_selection_methods = {}
for scaler in scalers:
    for method in methods:
        key = f"{scaler}_{method}"
        filename = f"final_{key}.csv"
        try:
            df = pd.read_csv(filename)
            feature_selection_methods[key] = df
            print(f"Loaded {filename}")
        except Exception as e:
            print(f"Error loading {filename}: {e}")


# === Define Models for Testing ===
def define_models():
    return {
        "SVM": SVC(kernel='linear', probability=True, random_state=42, max_iter=5000),
        "SVM_RBF": SVC(kernel='rbf', probability=True, random_state=42, max_iter=5000),
        "SVM_Poly": SVC(kernel='poly', probability=True, random_state=42, max_iter=5000),
        "L1LogisticRegression": LogisticRegression(penalty='l1', solver='liblinear',
                                                   class_weight='balanced', random_state=42, max_iter=1000),
        "LDA": LinearDiscriminantAnalysis()
    }

# Set the grid search scoring metric to maximize (traditionally f1 for classification)
grid_search_scoring = "f1"


# Train & Evaluate Models
def train_and_evaluate_models(X_df, method_name):
    print(f"\n\033[1;34m=== TESTING FEATURE SELECTION METHOD: {method_name} ===\033[0m")

    # Partial Undersampling
    df_combined = X_df.copy()
    counts = df_combined['Status'].value_counts()
    if len(counts) == 2:
        minority_class = counts.idxmin()
        majority_class = counts.idxmax()
        minority_count = counts.min()
        desired_ratio = 2
        df_minority = df_combined[df_combined['Status'] == minority_class]
        df_majority = df_combined[df_combined['Status'] == majority_class]
        sample_size = min(len(df_majority), desired_ratio * minority_count)
        df_majority_sampled = df_majority.sample(sample_size, random_state=42)
        df_undersampled = pd.concat([df_minority, df_majority_sampled])
    else:
        df_undersampled = df_combined
    print("Data shape after partial undersampling:", df_undersampled.shape)

    # Separate features and target
    X_features = df_undersampled.drop(columns=['Status'])
    y_target = df_undersampled['Status']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_target, test_size=0.2, random_state=42, stratify=y_target
    )
    print("Training set shape:", X_train.shape)
    print("Testing set shape:", X_test.shape)

    models = define_models()
    results_local = {}
    optimized_local = {}

    def plot_confusion_matrix(cm, title):
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title(title)
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

    def get_scoring(estimator):
        return 'neg_log_loss'

    for name, model in models.items():
        print(f"\n=== Training and Evaluating Model: {name} ({method_name}) ===")
        scoring_metric = get_scoring(model)

        # Learning Curve (Pre-Optimized)
        print(f"\n--- Learning Curve for initial {name} model ({method_name}) ---")
        plot_learning_curve(
            model,
            f"Initial {name} Learning Curve ({method_name})",
            X_train, y_train,
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring=scoring_metric
        )

        # Pre-Optimized Train vs. Test Error (Using Log Loss)
        print(f"\n--- Pre-Optimized {name} Train vs. Test Error ({method_name}) ---")
        try:
            model.fit(X_train, y_train)
            plot_train_test_error(
                model, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PreOptim", method_name=method_name,
                scoring=scoring_metric
            )
        except Exception as e:
            print(f"Error fitting pre-optimized {name}: {e}")

        # Pre-Optimized KFold CV Accuracy
        print(f"\n--- Pre-Optimized KFold CV Accuracy for {name} ({method_name}) ---")
        cv_pre = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        pre_fold_accuracies = []
        for fold, (train_idx, val_idx) in enumerate(cv_pre.split(X_train, y_train), 1):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            model.fit(X_fold_train, y_fold_train)
            acc = model.score(X_fold_val, y_fold_val)
            pre_fold_accuracies.append(acc)
            print(f"{name} (Pre-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
        mean_pre_acc = np.mean(pre_fold_accuracies)
        print(f"{name} (Pre-Optimized) - Mean CV Accuracy: {mean_pre_acc:.4f}")
        plt.figure(figsize=(6,4))
        plt.bar(range(1, len(pre_fold_accuracies)+1), pre_fold_accuracies, color="#2ca02c")
        plt.title(f"Pre-Optimized CV Accuracy per Fold for {name}")
        plt.xlabel("Fold Number")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()

        # Grid Search using Product of Precision & Recall
        try:
            if name == "L1LogisticRegression":
                param_grid = {"clf__class_weight": [{0: 1, 1: w} for w in [2, 3, 4, 5, 6]]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=product_pr_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name in ["SVM", "SVM_RBF"]:
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100]}
                if name == "SVM_RBF":
                    param_grid["clf__gamma"] = ['scale', 'auto']
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=product_pr_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "SVM_Poly":
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100],
                              "clf__degree": [2, 3, 4]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=product_pr_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "LDA":
                param_grid = {
                    "clf__solver": ["lsqr", "eigen"],
                    "clf__shrinkage": [0.3, 0.5, 0.7]
                }
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=product_pr_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                    n_jobs=-1, error_score='raise'
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            else:
                model_optimized = model

            optimized_local[name] = model_optimized

            # Learning Curve (Optimized)
            print(f"\n--- Learning Curve for optimized {name} model ({method_name}) ---")
            plot_learning_curve(
                model_optimized,
                f"Optimized {name} Learning Curve ({method_name})",
                X_train, y_train,
                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                scoring=scoring_metric
            )

            # Post-Optimized Train vs. Test Error (Using Log Loss)
            print(f"\n--- Post-Optimized {name} Train vs. Test Loss ({method_name}) ---")
            model_optimized.fit(X_train, y_train)
            plot_train_test_error(
                model_optimized, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PostOptim", method_name=method_name,
                scoring=scoring_metric
            )

            # Post-Optimized KFold CV Accuracy
            print(f"\n--- Post-Optimized KFold CV Accuracy for {name} ({method_name}) ---")
            cv_post = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            post_fold_accuracies = []
            for fold, (train_idx, val_idx) in enumerate(cv_post.split(X_train, y_train), 1):
                X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                model_optimized.fit(X_fold_train, y_fold_train)
                acc = model_optimized.score(X_fold_val, y_fold_val)
                post_fold_accuracies.append(acc)
                print(f"{name} (Post-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
            mean_post_acc = np.mean(post_fold_accuracies)
            print(f"{name} (Post-Optimized) - Mean CV Accuracy: {mean_post_acc:.4f}")
            plt.figure(figsize=(6,4))
            plt.bar(range(1, len(post_fold_accuracies)+1), post_fold_accuracies, color="#2ca02c")
            plt.title(f"Post-Optimized CV Accuracy per Fold for {name}")
            plt.xlabel("Fold Number")
            plt.ylabel("Accuracy")
            plt.ylim(0, 1)
            plt.tight_layout()
            plt.show()

            # Evaluate on Test Set: Unadjusted and Adjusted Confusion Matrices
            if hasattr(model_optimized, "predict_proba"):
                # Compute unadjusted predictions and confusion matrix.
                y_pred = model_optimized.predict(X_test)
                cm_unadjusted = confusion_matrix(y_test, y_pred)
                report_unadjusted = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                print(f"\nUnadjusted Classification Report for {name} ({method_name}):")
                print(classification_report(y_test, y_pred, zero_division=0))
                plot_confusion_matrix(cm_unadjusted, f"Unadjusted Confusion Matrix: {name} ({method_name})")

                # Now apply threshold adjustment using precision-recall.
                y_scores = model_optimized.predict_proba(X_test)[:, 1]
                best_threshold = plot_precision_recall_and_threshold(
                    y_test, y_scores, model_name=name, method_name=method_name
                )
                y_pred_thresh = (y_scores >= best_threshold).astype(int)
                report_adjusted = classification_report(y_test, y_pred_thresh, zero_division=0, output_dict=True)
                print(f"\nAdjusted Classification Report at threshold {best_threshold:.2f} for {name} ({method_name}):")
                print(classification_report(y_test, y_pred_thresh, zero_division=0))
                cm_adjusted = confusion_matrix(y_test, y_pred_thresh)
                plot_confusion_matrix(cm_adjusted, f"Final Adjusted Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix_unadjusted": cm_unadjusted,
                    "classification_report_unadjusted": report_unadjusted,
                    "confusion_matrix_adjusted": cm_adjusted,
                    "classification_report_adjusted": report_adjusted,
                    "FeatureSelectionMethod": method_name
                }
            else:
                y_pred = model_optimized.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                report = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                plot_confusion_matrix(cm, f"Final Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix": cm,
                    "classification_report_unadjusted": report,
                    "FeatureSelectionMethod": method_name
                }

        except Exception as e:
            print(f"Error evaluating {name} on {method_name}: {e}")
            results_local[name] = {"error": str(e), "FeatureSelectionMethod": method_name}

    return {"performance": results_local, "optimized_models": optimized_local}


# === STEP 3: Run Evaluations on Each Feature Selection Method ===
evaluation_results = {}
for method_key, df in feature_selection_methods.items():
    evaluation_results[method_key] = train_and_evaluate_models(df, method_key)

print("\n=== Model Evaluation Completed for All Feature Selection Methods ===")

# Save evaluation results
joblib.dump(evaluation_results, 'evaluation_results.pkl')
print("Evaluation results saved to 'evaluation_results.pkl'")

optimized_models = {}
for method_key, result in evaluation_results.items():
    optimized_models[method_key] = result.get("optimized_models", {})
joblib.dump(optimized_models, 'optimized_models.pkl')
print("Optimized models saved to 'optimized_models.pkl'")


In [ ]:
lol = "extracted_features_with_prep.csv"
lol2 = pd.read_csv(lol)
lol2.columns

files.download("final_Standard_RFECV_LR.csv")

Minimize diff grid search

In [ ]:
# Suppress warnings for undefined metrics and deprecated messages.
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", message="`use_label_encoder` is deprecated")


# Maximizing Product of Precision & Recall while Minimizing Their Difference
def composite_precision_recall(y_true, y_pred, lambda_val=1.5):
    p = precision_score(y_true, y_pred, zero_division=0)
    r = recall_score(y_true, y_pred, zero_division=0)
    return (p + r) - lambda_val * abs(p - r)

composite_scorer = make_scorer(composite_precision_recall, greater_is_better=True)

# Updated Learning Curve Function using Training Loss (Log Loss)
def plot_learning_curve(estimator, title, X, y, cv, scoring):
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1
    )

    # Convert negative log loss to positive loss if needed.
    if scoring == 'neg_log_loss':
        train_loss = -np.mean(train_scores, axis=1)
        val_loss = -np.mean(val_scores, axis=1)
    else:
        train_loss = 1 - np.mean(train_scores, axis=1)
        val_loss = 1 - np.mean(val_scores, axis=1)

    plt.figure(figsize=(8, 6))
    plt.plot(train_sizes, train_loss, 'o-', color="r", label="Training Loss")
    plt.plot(train_sizes, val_loss, 'o-', color="g", label="Validation Loss")
    plt.xlabel("Number of Training Samples")
    plt.ylabel("Log Loss")
    plt.title(title)
    plt.legend(loc="best")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return train_sizes, train_loss, val_loss

# Flatten nested dictionaries for CSV storage
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


# Train-vs-Test Error Function Using Log Loss
def plot_train_test_error(model, X_train, y_train, X_test, y_test, model_name, method_name, scoring="neg_log_loss"):

    try:
        y_train_pred_prob = model.predict_proba(X_train)
        y_test_pred_prob  = model.predict_proba(X_test)
    except Exception as e:
        print("Model does not support predict_proba, falling back to dummy conversion.")
        y_train_pred = model.predict(X_train)
        y_test_pred  = model.predict(X_test)
        y_train_pred_prob = np.vstack([1 - y_train_pred, y_train_pred]).T
        y_test_pred_prob  = np.vstack([1 - y_test_pred, y_test_pred]).T

    train_loss = log_loss(y_train, y_train_pred_prob)
    test_loss  = log_loss(y_test, y_test_pred_prob)
    metric_label = "Log Loss"

    loss_difference = test_loss - train_loss
    loss_ratio = test_loss / train_loss if train_loss != 0 else float('inf')

    print(f"{model_name} ({method_name}):")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Testing Loss:  {test_loss:.4f}")
    print(f"  Loss Difference (Test - Train): {loss_difference:.4f}")
    print(f"  Loss Ratio (Test / Train): {loss_ratio:.2f}")

    if loss_ratio > 1.2:
        print("  Warning: Test Loss is significantly higher than Training Loss. Possible overfitting detected.")
    else:
        print("  Training and Testing Loss are similar. Overfitting is less likely.")

    plt.figure(figsize=(5,4))
    plt.bar(["Train Loss", "Test Loss"], [train_loss, test_loss], color=["#1f77b4", "#ff7f0e"])
    plt.title(f"Training vs. Testing Loss\n{model_name} ({method_name})")
    plt.ylabel(metric_label)
    plt.grid(axis='y', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Finding the Threshold Where Precision - Recall is Minimized
def plot_precision_recall_and_threshold(y_true, y_scores, model_name, method_name):

    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    # Compute F1 scores for each threshold
    f1_scores = 2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-10)
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx]
    print(f"Optimal threshold for {model_name} ({method_name}) based on max F1 score: {best_threshold:.2f}")
    avg_precision = average_precision_score(y_true, y_scores)

    plt.figure(figsize=(8,6))
    plt.plot(recall, precision, label=f'AP = {avg_precision:.2f}')
    plt.scatter(recall[best_idx], precision[best_idx], color='red',
                label=f'Optimal (F1 = {f1_scores[best_idx]:.2f})')
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title(f"Precision-Recall Curve: {model_name} ({method_name})\n(Maximizing F1 Score)")
    plt.legend(loc='lower left')
    plt.tight_layout()
    plt.show()
    return best_threshold




# Load Feature Selection Outputs
print("=== Step 1: Load Feature Selection Outputs ===")
scalers = ["Standard", "MinMax", "Robust"]
methods = ["RFECV_LR", "PCA", "MutualInfo", "Full"]

feature_selection_methods = {}
for scaler in scalers:
    for method in methods:
        key = f"{scaler}_{method}"
        filename = f"final_{key}.csv"
        try:
            df = pd.read_csv(filename)
            feature_selection_methods[key] = df
            print(f"Loaded {filename}")
        except Exception as e:
            print(f"Error loading {filename}: {e}")


# Define Models
def define_models():
    return {
        "SVM": SVC(kernel='linear', probability=True, random_state=42, max_iter=5000),
        "SVM_RBF": SVC(kernel='rbf', probability=True, random_state=42, max_iter=5000),
        "SVM_Poly": SVC(kernel='poly', probability=True, random_state=42, max_iter=5000),
        "L1LogisticRegression": LogisticRegression(penalty='l1', solver='liblinear',
                                                   class_weight='balanced', random_state=42, max_iter=1000),
        "LDA": LinearDiscriminantAnalysis()
    }

# Set the grid search scoring metric to maximize
grid_search_scoring = "f1"


# Train & Evaluate Models
def train_and_evaluate_models(X_df, method_name):
    print(f"\n\033[1;34m=== TESTING FEATURE SELECTION METHOD: {method_name} ===\033[0m")

    # PARTIAL UNDERSAMPLING
    df_combined = X_df.copy()
    counts = df_combined['Status'].value_counts()
    if len(counts) == 2:
        minority_class = counts.idxmin()
        majority_class = counts.idxmax()
        minority_count = counts.min()
        desired_ratio = 2
        df_minority = df_combined[df_combined['Status'] == minority_class]
        df_majority = df_combined[df_combined['Status'] == majority_class]
        sample_size = min(len(df_majority), desired_ratio * minority_count)
        df_majority_sampled = df_majority.sample(sample_size, random_state=42)
        df_undersampled = pd.concat([df_minority, df_majority_sampled])
    else:
        df_undersampled = df_combined
    print("Data shape after partial undersampling:", df_undersampled.shape)

    # Separate features and target
    X_features = df_undersampled.drop(columns=['Status'])
    y_target = df_undersampled['Status']

    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_features, y_target, test_size=0.2, random_state=42, stratify=y_target
    )
    print("Training set shape:", X_train.shape)
    print("Testing set shape:", X_test.shape)

    models = define_models()
    results_local = {}
    optimized_local = {}

    def plot_confusion_matrix(cm, title):
        plt.figure(figsize=(6,4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
        plt.title(title)
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.show()

    def get_scoring(estimator):
        return 'neg_log_loss'

    for name, model in models.items():
        print(f"\n=== Training and Evaluating Model: {name} ({method_name}) ===")
        scoring_metric = get_scoring(model)

        # Learning Curve (Pre-Optimized)
        print(f"\n--- Learning Curve for initial {name} model ({method_name}) ---")
        plot_learning_curve(
            model,
            f"Initial {name} Learning Curve ({method_name})",
            X_train, y_train,
            cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
            scoring=scoring_metric
        )

        # Pre-Optimized Train vs. Test Error (Using Log Loss)
        print(f"\n--- Pre-Optimized {name} Train vs. Test Error ({method_name}) ---")
        try:
            model.fit(X_train, y_train)
            plot_train_test_error(
                model, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PreOptim", method_name=method_name,
                scoring=scoring_metric
            )
        except Exception as e:
            print(f"Error fitting pre-optimized {name}: {e}")

        # Pre-Optimized KFold CV Accuracy
        print(f"\n--- Pre-Optimized KFold CV Accuracy for {name} ({method_name}) ---")
        cv_pre = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        pre_fold_accuracies = []
        for fold, (train_idx, val_idx) in enumerate(cv_pre.split(X_train, y_train), 1):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            model.fit(X_fold_train, y_fold_train)
            acc = model.score(X_fold_val, y_fold_val)
            pre_fold_accuracies.append(acc)
            print(f"{name} (Pre-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
        mean_pre_acc = np.mean(pre_fold_accuracies)
        print(f"{name} (Pre-Optimized) - Mean CV Accuracy: {mean_pre_acc:.4f}")
        plt.figure(figsize=(6,4))
        plt.bar(range(1, len(pre_fold_accuracies)+1), pre_fold_accuracies, color="#2ca02c")
        plt.title(f"Pre-Optimized CV Accuracy per Fold for {name}")
        plt.xlabel("Fold Number")
        plt.ylabel("Accuracy")
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()

        # Grid Search using the Composite Precision-Recall Scorer
        try:
            if name == "L1LogisticRegression":
                param_grid = {"clf__class_weight": [{0: 1, 1: w} for w in [2, 3, 4, 5, 6]]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "RandomForest":
                param_grid = {
                    "clf__class_weight": [{0: 1, 1: w} for w in [2, 3, 4, 5, 6]],
                    "clf__max_depth": [None, 5, 10],
                    "clf__min_samples_leaf": [1, 3, 5],
                    "clf__random_state": [7, 42, 100, 2025]
                }
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name in ["SVM", "SVM_RBF"]:
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100]}
                if name == "SVM_RBF":
                    param_grid["clf__gamma"] = ['scale', 'auto']
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "SVM_Poly":
                param_grid = {"clf__C": [0.01, 0.1, 1, 10, 100],
                              "clf__degree": [2, 3, 4]}
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "XGBoost":
                param_grid = {
                    "clf__learning_rate": [0.01, 0.1, 0.2],
                    "clf__max_depth": [3, 5, 7]
                }
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), n_jobs=-1
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            elif name == "LDA":
                param_grid = {
                    "clf__solver": ["lsqr", "eigen"],
                    "clf__shrinkage": [0.3, 0.5, 0.7]
                }
                pipeline = Pipeline([('clf', model)])
                grid = GridSearchCV(
                    pipeline, param_grid, scoring=composite_scorer,
                    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                    n_jobs=-1, error_score='raise'
                )
                grid.fit(X_train, y_train)
                print(f"Best parameters for {name}: {grid.best_params_}")
                model_optimized = grid.best_estimator_.named_steps['clf']

            else:
                model_optimized = model

            optimized_local[name] = model_optimized

            # Learning Curve (Optimized)
            print(f"\n--- Learning Curve for optimized {name} model ({method_name}) ---")
            plot_learning_curve(
                model_optimized,
                f"Optimized {name} Learning Curve ({method_name})",
                X_train, y_train,
                cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                scoring=scoring_metric
            )

            # Post-Optimized Train vs. Test Error (Using Log Loss)
            print(f"\n--- Post-Optimized {name} Train vs. Test Loss ({method_name}) ---")
            model_optimized.fit(X_train, y_train)
            plot_train_test_error(
                model_optimized, X_train, y_train, X_test, y_test,
                model_name=f"{name}_PostOptim", method_name=method_name,
                scoring=scoring_metric
            )

            # Post-Optimized KFold CV Accuracy
            print(f"\n--- Post-Optimized KFold CV Accuracy for {name} ({method_name}) ---")
            cv_post = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            post_fold_accuracies = []
            for fold, (train_idx, val_idx) in enumerate(cv_post.split(X_train, y_train), 1):
                X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                model_optimized.fit(X_fold_train, y_fold_train)
                acc = model_optimized.score(X_fold_val, y_fold_val)
                post_fold_accuracies.append(acc)
                print(f"{name} (Post-Optimized) - Fold {fold} Accuracy: {acc:.4f}")
            mean_post_acc = np.mean(post_fold_accuracies)
            print(f"{name} (Post-Optimized) - Mean CV Accuracy: {mean_post_acc:.4f}")
            plt.figure(figsize=(6,4))
            plt.bar(range(1, len(post_fold_accuracies)+1), post_fold_accuracies, color="#2ca02c")
            plt.title(f"Post-Optimized CV Accuracy per Fold for {name}")
            plt.xlabel("Fold Number")
            plt.ylabel("Accuracy")
            plt.ylim(0, 1)
            plt.tight_layout()
            plt.show()

            # Unadjusted and Adjusted Confusion Matrices
            if hasattr(model_optimized, "predict_proba"):
                # Compute unadjusted predictions and confusion matrix.
                y_pred = model_optimized.predict(X_test)
                cm_unadjusted = confusion_matrix(y_test, y_pred)
                report_unadjusted = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                print(f"\nUnadjusted Classification Report for {name} ({method_name}):")
                print(classification_report(y_test, y_pred, zero_division=0))
                plot_confusion_matrix(cm_unadjusted, f"Unadjusted Confusion Matrix: {name} ({method_name})")

                # Compute adjusted predictions based on precision-recall threshold.
                y_scores = model_optimized.predict_proba(X_test)[:, 1]
                best_threshold = plot_precision_recall_and_threshold(
                    y_test, y_scores, model_name=name, method_name=method_name
                )
                y_pred_thresh = (y_scores >= best_threshold).astype(int)
                report_adjusted = classification_report(y_test, y_pred_thresh, zero_division=0, output_dict=True)
                print(f"\nAdjusted Classification Report at threshold {best_threshold:.2f} for {name} ({method_name}):")
                print(classification_report(y_test, y_pred_thresh, zero_division=0))
                cm_adjusted = confusion_matrix(y_test, y_pred_thresh)
                plot_confusion_matrix(cm_adjusted, f"Final Adjusted Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix_unadjusted": cm_unadjusted,
                    "classification_report_unadjusted": report_unadjusted,
                    "confusion_matrix_adjusted": cm_adjusted,
                    "classification_report_adjusted": report_adjusted,
                    "FeatureSelectionMethod": method_name
                }
            else:
                y_pred = model_optimized.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                report = classification_report(y_test, y_pred, zero_division=0, output_dict=True)
                plot_confusion_matrix(cm, f"Final Confusion Matrix: {name} ({method_name})")
                results_local[name] = {
                    "confusion_matrix": cm,
                    "classification_report_unadjusted": report,
                    "FeatureSelectionMethod": method_name
                }

        except Exception as e:
            print(f"Error evaluating {name} on {method_name}: {e}")
            results_local[name] = {"error": str(e), "FeatureSelectionMethod": method_name}

    return {"performance": results_local, "optimized_models": optimized_local}


# Run Evaluations on Each Feature Selection Method
evaluation_results = {}
for method_key, df in feature_selection_methods.items():
    evaluation_results[method_key] = train_and_evaluate_models(df, method_key)

print("\n=== Model Evaluation Completed for All Feature Selection Methods ===")

# Save evaluation results (performance metrics and optimized models) for later use.
joblib.dump(evaluation_results, 'evaluation_results.pkl')
print("Evaluation results saved to 'evaluation_results.pkl'")

optimized_models = {}
for method_key, result in evaluation_results.items():
    optimized_models[method_key] = result.get("optimized_models", {})
joblib.dump(optimized_models, 'optimized_models.pkl')
print("Optimized models saved to 'optimized_models.pkl'")


In [ ]:
lol = "final_Standard_Allowed.csv"
lol2 = pd.read_csv(lol)
lol2.columns

Characterizations

sources:

Man whitney for ranking:
https://pubmed.ncbi.nlm.nih.gov/25555756/ </br>
https://pmc.ncbi.nlm.nih.gov/articles/PMC10973702/?utm_source=chatgpt.com#hbm26555-sec-0026</br>
https://pmc.ncbi.nlm.nih.gov/articles/PMC3536906/?utm_source=chatgpt.com#sec23</br>







In [ ]:
# Load the feature ranking analysis file
analysis_file = "final_Standard_Allowed_feature_analysis.csv"
features_df = pd.read_csv(analysis_file)

# Select the relevant columns for in-depth normality and difference analysis:
features_df = features_df[["Feature", "Test_PValue", "Test_Statistic", "Shapiro_PValue"]]

# Sort features by the Mann–Whitney p-value (lower p-value => stronger evidence of difference)
sorted_by_diff = features_df.sort_values(by="Test_PValue", ascending=True)
sorted_by_diff['Decision'] = sorted_by_diff["Test_PValue"].apply(
    lambda x: "Reject Null (Feature Differs)" if x < 0.05 else "Fail to Reject Null"
)

# Sort features by the Mann–Whitney test statistic (higher statistic => stronger separation)
sorted_by_separability = features_df.sort_values(by="Test_Statistic", ascending=False)

# Mann–Whitney U Test p-values
plt.figure(figsize=(10, 5))
plt.barh(sorted_by_diff["Feature"], sorted_by_diff["Test_PValue"], color='blue', alpha=0.7)
plt.xlabel("Mann–Whitney U Test p-value")
plt.ylabel("Feature")
plt.title("Mann–Whitney U Test p-values\n(Lower p-value indicates stronger evidence of group differences)")
# Add vertical line indicating p = 0.05 threshold
plt.axvline(0.05, color='red', linestyle='--', label='p = 0.05')
plt.legend()
plt.gca().invert_yaxis()  # highest significance at top
plt.tight_layout()
plt.show()

# Mann–Whitney U Test Statistic
plt.figure(figsize=(10, 5))
plt.barh(sorted_by_separability["Feature"], sorted_by_separability["Test_Statistic"], color='green', alpha=0.7)
plt.xlabel("Mann–Whitney U Test Statistic")
plt.ylabel("Feature")
plt.title("Mann–Whitney U Test Statistic\n(Higher statistic indicates stronger group separability)")
plt.gca().invert_yaxis()  # highest statistic at top
plt.tight_layout()
plt.show()

# Shapiro–Wilk Normality Test p-values (with log scaling on the x-axis)
plt.figure(figsize=(10, 5))
plt.barh(sorted_by_diff["Feature"], sorted_by_diff["Shapiro_PValue"], color='orange', alpha=0.7)
plt.xlabel("Shapiro–Wilk Test p-value (Log Scale)")
plt.ylabel("Feature")
plt.title("Shapiro–Wilk Normality Test p-values\n(Lower p-value indicates deviation from normality)")
plt.xscale("log")

# Add vertical line indicating p = 0.05 threshold
plt.axvline(0.05, color='red', linestyle='--', label='p = 0.05')
plt.legend()
plt.gca().invert_yaxis()  # features with lowest p-value at top
plt.tight_layout()
plt.show()

# Correlation Heatmap of Analysis Metrics
plt.figure(figsize=(6, 4))
numeric_cols = features_df.select_dtypes(include=[np.number]).columns
sns.heatmap(features_df[numeric_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap of Analysis Metrics\n(Mann–Whitney and Shapiro–Wilk)")
plt.tight_layout()
plt.show()

# Pairplot of Analysis Metrics
sns.pairplot(features_df, diag_kind='kde')
plt.suptitle("Pairplot of Analysis Metrics\n(Mann–Whitney p-value, Test Statistic, Shapiro–Wilk p-value)", y=1.02)
plt.show()

# Scatter Plot of Mann–Whitney Metrics
plt.figure(figsize=(10, 6))
scatter = plt.scatter(features_df["Test_PValue"], features_df["Test_Statistic"],
                      c=features_df["Shapiro_PValue"], cmap='viridis', s=100, alpha=0.7)
plt.xlabel("Mann–Whitney U Test p-value")
plt.ylabel("Mann–Whitney U Test Statistic")
plt.title("Scatter Plot: Mann–Whitney p-value vs. Test Statistic\n(Colored by Shapiro–Wilk p-value)")
plt.colorbar(scatter, label="Shapiro–Wilk p-value")
plt.tight_layout()
plt.show()

# Violin Plots of Analysis Metrics
plt.figure(figsize=(12,6))
sns.violinplot(data=features_df[["Test_PValue", "Test_Statistic", "Shapiro_PValue"]])
plt.title("Violin Plots of Analysis Metrics\n(Distribution of Mann–Whitney and Shapiro–Wilk p-values/test statistics)")
plt.tight_layout()
plt.show()

# Display
print("=== Sorted by Mann–Whitney p-value (Lowest p-value = strongest evidence of difference) ===")
display(sorted_by_diff)

print("\n=== Sorted by Mann–Whitney Test Statistic (Highest = strongest group separability) ===")
display(sorted_by_separability)


In [ ]:
lol = "final_Standard_Allowed.csv"
lol2 = pd.read_csv(lol)
lol2.columns

***Ranking with PCA***

sources:<br>
basics - https://arxiv.org/pdf/1404.1100 <br>
ranking formula - https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0183997

In [ ]:
filename = "final_Robust_MutualInfo.csv"

# Check if the file exists
if not os.path.isfile(filename):
    print(f"File not found: {filename}")
else:
    # Load the data
    df = pd.read_csv(filename)

    features = df.select_dtypes(include=[np.number])
    if 'Status' in features.columns:
        features = features.drop(columns=['Status'])

    if features.empty:
        print(f"No numeric features to process in {filename}")
    else:

        X_scaled = features
        X_scaled = pd.DataFrame(X_scaled, columns=features.columns)

        # PCA for top 90% variance
        pca = PCA(n_components=0.90)
        pca.fit(X_scaled)
        n_components = pca.n_components_
        print(f"Number of PCA components selected to explain 90% variance: {n_components}")

        loadings = pd.DataFrame(
            pca.components_.T,
            index=features.columns,
            columns=[f"PC{i+1}" for i in range(n_components)]
        )

        feature_importance = (loadings**2).sum(axis=1)
        ranking = feature_importance.sort_values(ascending=False)

        print(f"\nPCA Feature Ranking for {filename} (using top 90% variance components):")
        print(ranking)

        # Plot cumulative explained variance (selected components)
        plt.figure(figsize=(8, 6))
        weights = pca.explained_variance_ratio_
        cum_var = np.cumsum(weights)
        plt.plot(range(1, n_components+1), cum_var, marker='o')
        plt.xticks(range(1, n_components+1))
        plt.xlabel("Principal Component")
        plt.ylabel("Cumulative Explained Variance")
        plt.title(f"Cumulative Explained Variance (90%)\n{filename}")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # PCA on all components to see full cumulative explained variance
        pca_full = PCA()
        pca_full.fit(X_scaled)

        full_weights = pca_full.explained_variance_ratio_
        full_cum_var = np.cumsum(full_weights)

        plt.figure(figsize=(10, 6))
        plt.plot(np.arange(1, len(full_cum_var)+1), full_cum_var, marker='o', color='purple')
        plt.xticks(np.arange(1, len(full_cum_var)+1))
        plt.xlabel("Principal Component")
        plt.ylabel("Cumulative Explained Variance")
        plt.title(f"Cumulative Explained Variance (All Components)\n{filename}")
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Feature Contributions per selected Principal Component
        for i in range(n_components):
            pc = f"PC{i+1}"
            pc_contrib = loadings[pc]**2
            pc_contrib_sorted = pc_contrib.sort_values(ascending=True)

            plt.figure(figsize=(10, max(5, 0.3 * len(pc_contrib_sorted))))
            plt.barh(pc_contrib_sorted.index, pc_contrib_sorted.values, color='lightgreen')
            plt.xlabel("Squared Loading (Variance Contribution)")
            plt.title(f"Feature Contribution for {pc}\n{filename}")
            plt.tight_layout()
            plt.show()

        # Total Variance Contribution by Feature (Communality)
        plt.figure(figsize=(10, max(5, 0.3 * len(ranking))))
        plt.barh(ranking.index, ranking.values, color='salmon')
        plt.xlabel("Total Variance Contribution (Communality)")
        plt.title(f"Total Variance Contribution by Feature (Top Components)\n{filename}")
        plt.tight_layout()
        plt.show()

        # PCA ranking (communality scores)
        ranking_sorted = ranking.sort_values(ascending=True)
        plt.figure(figsize=(10, max(5, 0.3 * len(ranking_sorted))))
        plt.barh(ranking_sorted.index, ranking_sorted.values, color='skyblue')
        plt.xlabel("Loading Score (Communality)")
        plt.title(f"PCA Feature Ranking\n{filename}")
        plt.tight_layout()
        plt.show()


In [ ]:
#files.download("final_Standard_RFECV_LR.csv")
"""files.download("final_Robust_RFECV_LR.csv")
files.download("final_Robust_RFECV_SVM.csv")"""
pd.read_csv("final_Standard_Allowed.csv")
df.columns

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu

# Load your dataset.
df = pd.read_csv('final_Robust_MutualInfo.csv')

features = df.drop(columns=['Status'])
target = df['Status']

# Initialize a list to store the U test results.
results = []

# Loop over each feature, performing the Mann–Whitney U test between the two classes.
for col in features.columns:
    group0 = features[target == 0][col]
    group1 = features[target == 1][col]
    # Use a two-sided test.
    U_statistic, p_value = mannwhitneyu(group0, group1, alternative='two-sided')
    results.append({'Feature': col, 'U_statistic': U_statistic, 'p_value': p_value})

# Convert results to a DataFrame and sort by p-value (lower is better).
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='p_value')

print("Feature ranking based on Mann–Whitney U test:")
print(results_df)

# Plot the ranking of features by their p-value.
plt.figure(figsize=(10, 6))
sns.barplot(x='p_value', y='Feature', data=results_df, palette='viridis')
plt.xlabel('Mann–Whitney U Test p-value')
plt.title('Feature Ranking using U‑Filter (Mann–Whitney U Test)')
plt.tight_layout()
plt.show()


In [ ]:
lol = "final_Standard_Allowed.csv"
lol2 = pd.read_csv(lol)
lol2.columns

files.download(lol)

interclass correlation

In [ ]:
df = pd.read_csv("final_Robust_MutualInfo.csv")

features = df.drop(columns=["Status"])
target = df["Status"]

# Split the data into separate DataFrames for each class.
df_class0 = df[df["Status"] == 0].drop(columns=["Status"])
df_class1 = df[df["Status"] == 1].drop(columns=["Status"])

# Compute the correlation matrix for each class.
corr_class0 = df_class0.corr()
corr_class1 = df_class1.corr()

# Define annotation settings for clarity
annot_kws = {"fontsize": 10}

# Plot the correlation matrix for Class 0 with a larger figure.
plt.figure(figsize=(16, 12))
sns.heatmap(corr_class0, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Pearsons Correlation Matrix for Class 0 after feature selection", fontsize=16)
plt.tight_layout()
plt.show()

# Plot the correlation matrix for Class 1 with a larger figure.
plt.figure(figsize=(16, 12))
sns.heatmap(corr_class1, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Pearsons Correlation Matrix for Class 1 after feature selection", fontsize=16)
plt.tight_layout()
plt.show()

# Compute the difference between the two correlation matrices.
diff_corr = corr_class0 - corr_class1

# Plot the difference in correlations with a larger figure.
plt.figure(figsize=(16, 12))
sns.heatmap(diff_corr, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Difference in Correlation (Class 0 - Class 1)", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
lol = "final_Standard_Allowed.csv"
lol2 = pd.read_csv(lol)
lol2.columns

spearman on non normal

In [ ]:
df = pd.read_csv("final_Robust_MutualInfo.csv")
df.columns

In [ ]:
df = pd.read_csv("final_Robust_MutualInfo.csv")
lol2 = ['Height', 'Active Playing Years', 'GrayLevelNonUniformity', 'Q-Angle',
       'Mean', 'Gender', 'Dissimilarity_0', 'Homogeneity_0', 'Hu6', 'Status']
non_normal_features = lol2

df_class0 = df[df["Status"] == 0][non_normal_features]
df_class1 = df[df["Status"] == 1][non_normal_features]

corr_class0 = df_class0.corr(method="spearman")
corr_class1 = df_class1.corr(method="spearman")

annot_kws = {"fontsize": 10}

plt.figure(figsize=(16, 12))
sns.heatmap(corr_class0, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Spearman Correlation Matrix (Non-Normal) - Class 0", fontsize=16)
plt.tight_layout()
plt.show()

plt.figure(figsize=(16, 12))
sns.heatmap(corr_class1, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Spearman Correlation Matrix (Non-Normal) - Class 1", fontsize=16)
plt.tight_layout()
plt.show()

diff_corr = corr_class0 - corr_class1

plt.figure(figsize=(16, 12))
sns.heatmap(diff_corr, annot=True, fmt=".2f", cmap="coolwarm", annot_kws=annot_kws)
plt.title("Difference in Spearman Correlation (Class 0 - Class 1)", fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
lol = "final_Standard_RFECV_LR.csv"
lol2 = pd.read_csv(lol)
lol2.columns

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('final_Standard_RFECV_LR.csv')

if 'Status' not in df.columns:
    raise ValueError("The dataset does not contain a 'Status' column.")

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Status' in numeric_cols:
    numeric_cols.remove('Status')

group0 = df[df['Status'] == 0]
group1 = df[df['Status'] == 1]

results = []
for col in numeric_cols:
    values0 = group0[col].dropna()
    values1 = group1[col].dropna()
    t_stat, p_value = ttest_ind(values0, values1, equal_var=False)
    results.append({'Feature': col, 't_statistic': t_stat, 'p_value': p_value})

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='p_value', ascending=True)

print("Independent Samples t-test Results:")
print(results_df)

# Plot p-values for visualization
plt.figure(figsize=(10, 6))
sns.barplot(x="p_value", y="Feature", data=results_df, palette="viridis")
plt.axvline(0.05, color="red", linestyle="--", label="Significance Threshold (p=0.05)")
plt.xlabel("p-value")
plt.title("Independent Samples t-test p-values for each Feature")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
lol = "final_Standard_RFECV_LR.csv"
lol2 = pd.read_csv(lol)
lol2.columns

In [ ]:
# Load dataset
df = pd.read_csv("final_Standard_RFECV_LR.csv")

# Ensure 'Status' column is present
if 'Status' not in df.columns:
    raise ValueError("The dataset does not contain a 'Status' column.")

# Identify numeric columns excluding 'Status'
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('Status')

# Split the dataset into two groups based on the binary target
group0 = df[df['Status'] == 0]
group1 = df[df['Status'] == 1]

# Conduct Welch's t-tests and store results
results = []
for col in numeric_cols:
    values0 = group0[col].dropna()
    values1 = group1[col].dropna()

    t_stat, p_value = ttest_ind(values0, values1, equal_var=False)

    results.append({'Feature': col, 't_statistic': t_stat, 'p_value': p_value})

# Create DataFrame for results
results_df = pd.DataFrame(results).sort_values(by='p_value')

# Save results to CSV
results_df.to_csv('ttest_results_selected_features.csv', index=False)

# Visualization
plt.figure(figsize=(10, 8))
sns.barplot(x="p_value", y="Feature", data=results_df, palette="viridis")
plt.axvline(0.05, color="red", linestyle="--", label="Significance Threshold (p=0.05)")
plt.xlabel("p-value")
plt.title("Feature Significance Based on Welch's t-test")
plt.legend()
plt.tight_layout()
plt.show()

# Save results to CSV
results_df.to_csv('ttest_results_RFECV_selected_features.csv', index=False)

print("Results saved to 'ttest_results_export.csv'")


In [ ]:
lol = "final_Standard_RFECV_LR.csv"
lol2 = pd.read_csv(lol)
lol2.columns

In [ ]:
files.download("final_Standard_MutualInfo_feature_analysis.csv")